In [1]:
import numpy as np
import pandas as pd
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn import *

/home/mohan/miniconda3/envs/dft/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mohan/miniconda3/envs/dft/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/mohan/miniconda3/envs/dft/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This modu

In [2]:
train = pd.read_csv('./train.csv.zip')
test = pd.read_csv('./test.csv.zip')

FileNotFoundError: [Errno 2] No such file or directory: './train.csv.zip'

In [26]:
print(train.shape, test.shape)

(4459, 4993) (49342, 4992)


In [27]:
train.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [28]:
only_one = train.columns[train.nunique() == 1]
train.drop(only_one, axis = 1, inplace = True)
test.drop(only_one, axis = 1, inplace = True)

In [29]:
print(train.shape, test.shape)

(4459, 4737) (49342, 4736)


In [31]:


col = [c for c in train.columns if c not in ['ID', 'target']]
xtrain = train[col].copy().values
target = train['target'].values

In [19]:
from PIL import Image, ImageDraw, ImageColor
im = Image.new('RGBA', xtrain.shape)
wh = ImageColor.getrgb('white')
re = ImageColor.getrgb('red')
gr = ImageColor.getrgb('green')
ga = ImageColor.getrgb('gray')

for x in range(xtrain.shape[0]):
    for y in range(xtrain.shape[1]):
        if xtrain[x][y] == 0:
            im.putpixel((x,y), wh)
        elif xtrain[x][y] == target[x]:
            im.putpixel((x,y), re)
        elif (np.abs(xtrain[x][y] - target[x]) / target[x]) < 0.05:
            im.putpixel((x,y), gr)
        else:
            im.putpixel((x,y), ga)
im.save('leak.bmp')

In [32]:
leak_col = []
for c in col:
    leak1 = np.sum((train[c]==train['target']).astype(int))
    leak2 = np.sum((((train[c] - train['target']) / train['target']) < 0.05).astype(int))
    if leak1 > 30 and leak2 > 3500:
        leak_col.append(c)
print(len(leak_col))

51


In [33]:
col = list(leak_col)
train = train[col +  ['ID', 'target']]
test = test[col +  ['ID']]

In [34]:
train["nz_mean"] = train[col].apply(lambda x: x[x!=0].mean(), axis=1)
train["nz_max"] = train[col].apply(lambda x: x[x!=0].max(), axis=1)
train["nz_min"] = train[col].apply(lambda x: x[x!=0].min(), axis=1)
train["ez"] = train[col].apply(lambda x: len(x[x==0]), axis=1)
train["mean"] = train[col].apply(lambda x: x.mean(), axis=1)
train["max"] = train[col].apply(lambda x: x.max(), axis=1)
train["min"] = train[col].apply(lambda x: x.min(), axis=1)

test["nz_mean"] = test[col].apply(lambda x: x[x!=0].mean(), axis=1)
test["nz_max"] = test[col].apply(lambda x: x[x!=0].max(), axis=1)
test["nz_min"] = test[col].apply(lambda x: x[x!=0].min(), axis=1)
test["ez"] = test[col].apply(lambda x: len(x[x==0]), axis=1)
test["mean"] = test[col].apply(lambda x: x.mean(), axis=1)
test["max"] = test[col].apply(lambda x: x.max(), axis=1)
test["min"] = test[col].apply(lambda x: x.min(), axis=1)
col += ['nz_mean', 'nz_max', 'nz_min', 'ez', 'mean', 'max', 'min']

/Users/ching/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ching/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ching/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [35]:
print(train.shape, test.shape)
print(col)

(4459, 60) (49342, 59)
['20aa07010', '87ffda550', '963a49cdc', '63c094ba4', '26fc93eb7', '0572565c2', '66ace2992', 'fb49e4212', '6619d81fc', '6eef030c1', 'fc99f9426', '1c71183bb', 'df838756c', 'b43a7cfd5', '024c577b9', '2ec5b290f', '44d5b820f', '0ff32eb98', '58e056e12', '241f0f867', '1931ccfdd', '58e2e02e6', '9fd594eec', 'fb0f5dbfe', 'd5fa73ead', 'f6eba969e', '91f701ba2', '8e4d0fe45', '703885424', '6c5c8869c', '2e103d632', '122c135ed', 'eeb9cd3aa', '324921c7b', '58232a6fb', '491b9ee45', 'd6bb78916', '70feb1494', 'adb64ff71', '62e59a501', '15ace8c9f', '5c6487af1', 'f190486d6', 'f74e8f13d', 'c5a231d81', 'e176a204a', '1702b5bf0', '190db8488', 'c47340d97', '6df033973', '23310aa6f', 'nz_mean', 'nz_max', 'nz_min', 'ez', 'mean', 'max', 'min']


In [36]:
train.head()

,20aa07010,87ffda550,963a49cdc,63c094ba4,26fc93eb7,0572565c2,66ace2992,fb49e4212,6619d81fc,6eef030c1,...,23310aa6f,ID,target,nz_mean,nz_max,nz_min,ez,mean,max,min
0,0.0,1300000.0,13200000.0,7100000.0,3205000.0,400000.0,0.0,0.0,400000.0,900000.0,...,0.0,000d6aaf2,38000000.0,5.457803e+06,28000000.0,400000.0,29,2.354346e+06,28000000.0,0.0
1,2200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,...,0.0,000fbd867,600000.0,4.130833e+06,16000000.0,800000.0,41,8.099673e+05,16000000.0,0.0
2,0.0,0.0,12000000.0,0.0,0.0,0.0,5500000.0,0.0,0.0,0.0,...,0.0,0027d6b71,10000000.0,7.833333e+06,12000000.0,5500000.0,48,4.607843e+05,12000000.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0028cbf45,2000000.0,1.650000e+06,2000000.0,1300000.0,49,6.470588e+04,2000000.0,0.0
4,2000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37662000.0,...,0.0,002a68644,14400000.0,9.277429e+06,37662000.0,1180000.0,44,1.273373e+06,37662000.0,0.0


In [39]:
for i in range(2, 100):
    train['index'+str(i)] = ((train.index + 2) % i == 0).astype(int)
    test['index'+str(i)] = ((test.index + 2) % i == 0).astype(int)
    col.append('index'+str(i))

In [41]:
sub3 = pd.read_csv('./leaky_submission.csv')
test = pd.merge(test, sub3, how='left', on='ID',)

In [42]:
from scipy.sparse import csr_matrix, vstack
train = train.replace(0, np.nan)
test = test.replace(0, np.nan)
train = pd.concat((train, test), axis=0, ignore_index=True)

In [43]:
test['target'] = 0.0
folds = 5
for fold in range(folds):
    x1, x2, y1, y2 = model_selection.train_test_split(train[col], np.log1p(train.target.values), test_size=0.20, random_state=fold)
    params = {'learning_rate': 0.02, 'max_depth': 7, 'boosting': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'is_training_metric': True, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'seed':fold}
    model = lgb.train(params, lgb.Dataset(x1, label=y1), 3000, lgb.Dataset(x2, label=y2), verbose_eval=200, early_stopping_rounds=100)
    test['target'] += np.expm1(model.predict(test[col], num_iteration=model.best_iteration))
test['target'] /= folds
test[['ID', 'target']].to_csv('submission.csv', index=False)


Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 0.850041
[400]	valid_0's rmse: 0.847941
Early stopping, best iteration is:
[442]	valid_0's rmse: 0.847622
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 0.82896
[400]	valid_0's rmse: 0.827591
Early stopping, best iteration is:
[336]	valid_0's rmse: 0.827404
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 0.832559
[400]	valid_0's rmse: 0.830785
Early stopping, best iteration is:
[435]	valid_0's rmse: 0.830492
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 0.825973
[400]	valid_0's rmse: 0.82456
Early stopping, best iteration is:
[432]	valid_0's rmse: 0.82433
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 0.832116
[400]	valid_0's rmse: 0.830204
Early stopping, best iteration is:
[454]	valid_0's rmse: 0.830111


In [110]:
-3%2


1

In [61]:
        keys = list(range(1,27))
        values = [chr(i) for i in range(65,91)]
        
        mydict = dict(zip(keys, values))

In [76]:
a = 4/2
(4/2).is_integer()

True